In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
# import warnings
# warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
% matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

In [ ]:

data = pd.read_csv('emails.csv')
data.head(5)


,Email No.,the,to,ect,and,for,of,a,you,hou,in,on,is,this,enron,i,be,that,will,have,with,your,at,we,s,are,it,by,com,as,from,gas,or,not,me,deal,if,meter,hpl,please,...,bold,catch,performing,accepted,matters,batch,continuing,winning,symbol,offsystem,decisions,produced,ended,greatest,degree,solmonson,imbalances,fall,fear,hate,fight,reallocated,debt,reform,australia,plain,prompt,remains,ifhsc,enhancements,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,18,21,13,0,1,61,4,2,0,0,2,0,12,9,95,4,3,3,3,12,3,1,21,1,12,0,1,0,0,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,4,2,0,0,0,8,0,0,0,0,0,0,2,0,2,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,1,5,9,2,0,16,2,0,0,1,1,0,2,1,36,3,1,2,0,2,3,0,10,2,5,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,3,12,2,2,0,30,8,0,0,2,0,0,7,0,19,2,4,2,0,4,1,2,6,0,6,0,0,3,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
from sklearn import preprocessing
data = data[650:]
X = data.drop(columns=['Prediction', 'Email No.'])
y = data['Prediction']
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X)
X = x_scaled
print(X)

[[0.53982301 0.54117647 0.02915452 ... 0.         0.07142857 0.        ]
 [0.10619469 0.12941176 0.0058309  ... 0.         0.         0.        ]
 [0.00884956 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.01769912 0.08235294 0.         ... 0.         0.01190476 0.        ]
 [0.19469027 0.28235294 0.01166181 ... 0.         0.         0.        ]]


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4522 entries, 650 to 5171
Columns: 3002 entries, Email No. to Prediction
dtypes: int64(3001), object(1)
memory usage: 103.6+ MB


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=['Prediction', 'Email No.']).values, data['Prediction'], test_size=0.33, random_state=42)
y_train = np.asarray(y_train)
x_train = np.asarray(x_train)
y_test = np.asarray(y_test)
x_test = np.asarray(x_test)

In [ ]:

Check = LogisticRegression()
Check.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
accuracy = Check.score(x_test,y_test)
y_pred_test = Check.predict(x_test)
print(metrics.classification_report(y_test, y_pred_test))  

              precision    recall  f1-score   support

           0       0.98      0.97      0.98      1060
           1       0.93      0.96      0.95       433

    accuracy                           0.97      1493
   macro avg       0.96      0.97      0.96      1493
weighted avg       0.97      0.97      0.97      1493



Создадим свою модель логистической регрессии

In [ ]:
W = np.zeros(x_train.shape[1])
b = 0

def fit(X, Y,lr, iters):
    X1, Y1 = X.copy(), Y.copy()
    n = X.shape[0]
    W = np.zeros(X.shape[1])
    b = 0
    for i in range(iters):
        y_predicted = 1 / (1 + np.exp(-(np.dot(X1, W) + b)))
        W -= lr * ((1 / n) * np.dot(X.T, y_predicted - Y1))
        b -= lr * ((1 / n) * np.sum(y_predicted - Y))

def predict(X):
    probabilities = np.array(1 / (1 + np.exp(-(np.dot(X, W) + b))))
    y_pred = np.where(probabilities >= 0.5, 1, 0)
    return y_pred
    
x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=['Prediction', 'Email No.']).values, data['Prediction'], test_size=0.33, random_state=42)
y_train = np.asarray(y_train)
x_train = np.asarray(x_train)
y_test = np.asarray(y_test)
x_test = np.asarray(x_test)

x_train1 = np.apply_along_axis(lambda x: x/ x.std(), 0, x_train)
x_test1 = np.apply_along_axis(lambda x: x/ x.std(), 0, x_test)
fit(x_train1, y_train, 0.00001, 10000)
y_pred_test = predict(x_test1)
print(metrics.classification_report(y_test, y_pred_test))  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide


              precision    recall  f1-score   support

           0       0.71      1.00      0.83      1060
           1       0.00      0.00      0.00       433

    accuracy                           0.71      1493
   macro avg       0.35      0.50      0.42      1493
weighted avg       0.50      0.71      0.59      1493



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Теперь протестируем ее

Перейдем к решению задачи через Дерево Решений

In [ ]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier = classifier.fit(x_train, y_train)
# predict
y_pred = classifier.predict(x_test)
print(metrics.classification_report(y_test, y_pred))  

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      1060
           1       0.86      0.88      0.87       433

    accuracy                           0.92      1493
   macro avg       0.90      0.91      0.91      1493
weighted avg       0.92      0.92      0.92      1493



Теперь напишем свою реализацию Дерева Решений

In [ ]:
class Node():
    def __init__(
            self, 
            feature_index=None, 
            threshold=None, left=None,
            right=None, 
            info_gain=None, 
            value=None):

        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value

class decision_tree():
    def __init__(self, min_samples_split=2, max_depth=2):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, data, curr_depth=0):
        X, Y = data[:,:-1], data[:,-1]
        num_samples, num_features = np.shape(X)
        
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            best_split = self.get_best_split(data, num_samples, num_features)
            
            if best_split['info_gain'] > 0:
                left_subtree = self.build_tree(best_split['data_left'], curr_depth + 1)
                right_subtree = self.build_tree(best_split['data_right'], curr_depth + 1)
                return Node(best_split['feature_index'], best_split['threshold'], left_subtree, right_subtree, best_split['info_gain'])
        
        leaf_value = max(list(Y), key=list(Y).count)
        return Node(value=leaf_value)
    
    def get_best_split(self, data, num_samples, num_features):
        best_split = {}
        max_info_gain = -float('inf')
        
        for feature_index in range(num_features):
            feature_values = data[:, feature_index]
            for threshold in np.unique(feature_values):
                data_left, data_right = self.split(data, feature_index, threshold)
                if len(data_left) > 0 and len(data_right) > 0:
                    y, left_y, right_y = data[:, -1], data_left[:, -1], data_right[:, -1]
                    curr_info_gain = self.info_gain(y, left_y, right_y)
                    
                    if curr_info_gain > max_info_gain:
                        best_split['feature_index'] = feature_index
                        best_split['threshold'] = threshold
                        best_split['data_left'] = data_left
                        best_split['data_right'] = data_right
                        best_split['info_gain'] = curr_info_gain
                        max_info_gain = curr_info_gain
        return best_split
    

    def split(self, data, i, border):
        data_left = np.array([row for row in data if row[i] <= border])
        data_right = np.array([row for row in data if row[i] > border])
        return data_left, data_right
    
    def info_gain(self, parent, l_child, r_child):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        return self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))
    
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for c in class_labels:
            p = len(y[y == c]) / len(y)
            entropy += -p * np.log2(p)
        return entropy

    def fit(self, X, Y):
        self.root = self.build_tree(np.concatenate((X, Y), axis=1))
    
    def predict(self, X):
        return [self.prediction(x, self.root) for x in X]

    def prediction(self, x, tree):
        if tree.value != None: 
            return tree.value
        else:
            feature_val = x[tree.feature_index]
            if feature_val<=tree.threshold:
                return self.prediction(x, tree.left)
            else:
                return self.prediction(x, tree.right)

In [ ]:
params = [
    (2, 1),
    (2, 2),
    (2, 3),
    (2, 4),
    (3, 1),
    (3, 2),
    (3, 3),
    (3, 4),
    (4, 1),
    (4, 2),
    (4, 3),
    (4, 4)
]

min_samples_split, max_depth = 2,1
mytree = decision_tree(min_samples_split, max_depth)
mytree.fit(pd.DataFrame(x_train), pd.DataFrame(y_train))
print(metrics.classification_report(y_test, mytree.predict(x_test)))


              precision    recall  f1-score   support

           0       1.00      0.59      0.74      1060
           1       0.50      1.00      0.66       433

    accuracy                           0.71      1493
   macro avg       0.75      0.79      0.70      1493
weighted avg       0.85      0.71      0.72      1493



Мы видим довольно низкую точность, поменяем параметры дерева

In [ ]:
min_samples_split, max_depth = 3,4
mytree = decision_tree(min_samples_split, max_depth)
mytree.fit(pd.DataFrame(x_train), pd.DataFrame(y_train))
print(metrics.classification_report(y_test, mytree.predict(x_test)))

              precision    recall  f1-score   support

           0       0.84      0.97      0.90      1060
           1       0.89      0.55      0.68       433

    accuracy                           0.85      1493
   macro avg       0.87      0.76      0.79      1493
weighted avg       0.86      0.85      0.84      1493



Мы добились увеличения точности за счет увеличения глубины дерева

In [ ]:
class SVM_implementation:
    def __init__(self, lr, lambda_, iters):
        self.lr = lr
        self.lambda_ = lambda_
        self.iters = iters

    def fit(self, X_train, y_train):
        n = X_train.shape[0]
        self.W = np.zeros(X.shape[1])
        self.b = 0
        y_ = np.where(y <= 0, -1, 1)

        for i in range(self.iters):
            for j, x in enumerate(X_train):
                if y_[j] * ((x @ self.W) - self.b) >= 1:
                    self.W -= self.lr * (2 * self.lambda_ * self.W)
                else:
                    self.W -= self.lr * (2 * self.lambda_ * self.W - np.dot(x,y_[j]))
                    self.b -= self.lr * y_[j]

    def predict(self, X):
        res = np.dot(X, self.W) - self.b
        y_pred = np.where(res >= 0, 1, 0)
        return y_pred




In [ ]:

lr, lambda_, iters = 0.01, 0.01, 5000
my_svm = SVM_implementation(lr=lr, lambda_=lambda_, iters=iters)
my_svm.fit(x_train, y_train)
print(metrics.classification_report(y_test, my_svm.predict(x_test)))

              precision    recall  f1-score   support

           0       0.73      0.91      0.81      1060
           1       0.45      0.18      0.25       433

    accuracy                           0.70      1493
   macro avg       0.59      0.54      0.53      1493
weighted avg       0.65      0.70      0.65      1493



In [ ]:
Мы видим, что точность составила 59 процентов. Теперь опробуем поменять параметры

In [ ]:
lr, lambda_, iters = 0.001, 0.01, 5000
my_svm = SVM_implementation(lr=lr, lambda_=lambda_, iters=iters)
my_svm.fit(x_train, y_train)
print(metrics.classification_report(y_test, my_svm.predict(x_test)))

              precision    recall  f1-score   support

           0       0.71      0.94      0.81      1060
           1       0.31      0.07      0.11       433

    accuracy                           0.69      1493
   macro avg       0.51      0.50      0.46      1493
weighted avg       0.60      0.69      0.61      1493



In [ ]:
lr, lambda_, iters = 0.01, 0.01, 10000
my_svm = SVM_implementation(lr=lr, lambda_=lambda_, iters=iters)
my_svm.fit(x_train, y_train)
print(metrics.classification_report(y_test, my_svm.predict(x_test)))

              precision    recall  f1-score   support

           0       0.70      0.78      0.74      1060
           1       0.26      0.19      0.22       433

    accuracy                           0.61      1493
   macro avg       0.48      0.48      0.48      1493
weighted avg       0.57      0.61      0.59      1493



In [ ]:
from sklearn.svm import SVC

sklearn_svm_model = SVC()
sklearn_svm_model.fit(x_train, y_train)
y_pred = sklearn_svm_model.predict(x_test)
print(metrics.classification_report(y_test, y_pred)) 

              precision    recall  f1-score   support

           0       0.78      0.98      0.87      1060
           1       0.85      0.33      0.48       433

    accuracy                           0.79      1493
   macro avg       0.82      0.65      0.67      1493
weighted avg       0.80      0.79      0.75      1493



Мы видим, что библиотечная реализация является эффективней